In [1]:
import requests
import base64
from pprint import pprint

def get_data(namespace_id:str, height:int):
    r = requests.get(f'http://localhost:26659/namespaced_data/{namespace_id}/height/{height}')
    return dict(r.json())

In [2]:
from google.protobuf.message import DecodeError
from blob_pb2 import Blob

def is_valid_protobuf(data: bytes) -> bool:
    try:
        # Attempt to parse the data as a Blob message
        Blob().ParseFromString(data)
        return True
    except DecodeError:
        return False

In [3]:
from blob_pb2 import Blob

blob = Blob(address_owner="abcd123CELESTIA", name="pratham.tia")
data = blob.SerializeToString()

blob = Blob()
blob.ParseFromString(data)
print(blob.name)
print(blob.address)

pratham.tia
abcd123CELESTIA


In [4]:
def parse(data: str):
    decoded_data = base64.b64decode(data).decode('utf-8')
    return decoded_data

In [5]:
import binascii

hex_bytes = binascii.hexlify(data).decode('utf-8')

In [6]:
d = data.hex()

In [7]:
d

'0a0f6162636431323343454c4553544941120b7072617468616d2e746961'

In [8]:
binascii.unhexlify(d)

b'\n\x0fabcd123CELESTIA\x12\x0bpratham.tia'

In [9]:
hex_bytes

'0a0f6162636431323343454c4553544941120b7072617468616d2e746961'

In [10]:
def post_data(namespace_id, data, gas_limit=70000):
    d = data.hex()
    r = requests.post(f'http://localhost:26659/submit_pfd', json={"namespace_id":namespace_id,"data":d,"gas_limit":gas_limit})
    pprint(r.json()) 
    print(get_data('756f60cbe7bf5401',r.json()['height']))

In [11]:
get_data('756f60cbe7bf5401', 79192)

{'data': ['Cg9hYmNkMTIzQ0VMRVNUSUESC3ByYXRoYW0udGlh'], 'height': 79192}

In [ ]:
from blob_pb2 import Blob

post_data('756f60cbe7bf5401', data)

In [13]:
# {"type": "REGISTER_NAME", "payload": {"name": "Pratham"}, signature: "123"}
state = dict() # initial state is empty

# TODO: real implementation
def get_address(signature):
    # pretend like the address is the signature
    return signature

# can be improved
def is_valid_address(address):
    return address == str

def update_state(action):
    actionType = action.get("type")
    address = get_address(action.get("signature"))
    if actionType == "REGISTER_NAME":
        name = action.get("payload").get("name")
        state.setdefault(name, address)
    elif actionType == "TRANSFER_NAME":
        name = action.get("payload").get("name")
        recipient = action.get("payload").get("recipient")
        if state.get(name) is address and is_valid_address(recipient):
            state.set(name, recipient)

In [14]:
get_data("756f60cbe7bf5401",79043)

{'data': ['Cg9hYmNkMTIzQ0VMRVNUSUESC3ByYXRoYW0udGlh'], 'height': 79043}

In [15]:
def get_shares(namespace_id:str, height:int):
    r = requests.get(f'http://localhost:26659/namespaced_shares/{namespace_id}/height/{height}')
    return dict(r.json())

#

# Historical Syncing
We need to start from block 1 (or whatever block was the latest at the time that we launch this protocol) and process each block until the we get to the latest block.

In [16]:
get_shares("756f60cbe7bf5401",27071)

{'shares': ['dW9gy+e/VAEBGnsnUHJhdGhhbSc6ICdwcmFzb29uLmNlbCd9AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA='],
 'height': 27071}

In [17]:
my_namespace = '756f60cbe7bf5401'
initialHeight = 79010
latestHeight = 79015 # for example
for height in range(initialHeight, latestHeight):
    data_list = get_data(my_namespace, height)
    print("Trying", height)
    try:
        
        parsed_data = parse(data_list.get('data')[0])
        update_state(dict(eval(parsed_data)))
        print(is_valid_protobuf(parsed_data))
        print(data_list)
    except Exception:
        pass

Trying 79010
Trying 79011
Trying 79012
Trying 79013
Trying 79014


# Playing with Proto Buffers

pratham.tia
abcd123CELESTIA
